In [ ]:
import logging
import pprint

from molmass import Formula

from atmodeller import (
    ChemicalSpecies,
    EquilibriumModel,
    SpeciesNetwork,
    ThermodynamicState,
    debug_logger,
)
from atmodeller.thermodata.core import CondensateActivity

logger = debug_logger()
logger.setLevel(logging.INFO)

# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

# Gas Mixing

This notebook is available at `notebooks/gas_mixing.ipynb` and is easiest to obtain by downloading the source code.

In gas mixing experiments, a predefined gaseous mixture is injected into a chamber and allowed to equilibrate thermodynamically under controlled temperature-pressure conditions. In this example, we consider a gas mixture that equilibrates in the presence of solid carbon (graphite).

In [ ]:
# Enforce the stability of graphite
# Since in this example we do not provide carbon in the injected gas stream, we cannot solve for
# the stability of any carbon-bearing products because in order to do so requires specification of
# the mass of carbon in the system.
activity = CondensateActivity(1.0)
C_cr = ChemicalSpecies.create_condensed("C", activity=activity, solve_for_stability=False)

# Define allowable gas species at equilibrium
H2_g = ChemicalSpecies.create_gas("H2")
N2_g = ChemicalSpecies.create_gas("N2")
CH4_g = ChemicalSpecies.create_gas("CH4")
CHN_g = ChemicalSpecies.create_gas("CHN")
H_g = ChemicalSpecies.create_gas("H")

species = SpeciesNetwork((C_cr, H2_g, N2_g, CH4_g, CHN_g, H_g))

model = EquilibriumModel(species)

# Set the temperature and pressure
state = ThermodynamicState(temperature=1773.15, melt_fraction=0, pressure=1)

# Define the mole fractions of input gases
mole_fractions = {"H2": 0.5, "N2": 0.5}

# Define the composition of the input gas mixture by mass
mass_constraints = {key: value * Formula(key).mass for key, value in mole_fractions.items()}

# Solve
model.solve(state=state, mass_constraints=mass_constraints, solver_type="basic")

# The abundance of C cannot be solved for, so the output result for the number density of graphite
# should be ignored.
output = model.output
solution = output.quick_look()

logger.info("solution = %s", pprint.pformat(solution))

# Optionally dump the output to Excel
# output.to_excel("gas_mixing")